In [1]:
!python -V

Python 3.12.3


In [1]:
import math
import os.path
import re
import sys
from os import path

from loguru import logger

from app.config import config
from app.models import const
from app.models.schema import VideoParams, VideoConcatMode
from app.services import llm, material, voice, video, subtitle
from app.services import state as sm
from app.utils import utils

2024-05-09 23:33:58.054 | INFO     | app.config.config:load_config:23 - load config from file: /Users/roper/work/auto_video_generator/config.toml
2024-05-09 23:33:58.056 | INFO     | app.config.config:<module>:71 - MoneyPrinterTurbo v1.1.5


hello


In [2]:
params_dict = {
        "video_subject": "怎么快速赚到100万",
        "video_aspect": "9:16",
        "voice_name": "zh-CN-XiaoxiaoNeural",
        "enable_bgm": False,
        "font_name": "STHeitiMedium 黑体-中",
        "text_color": "#FFFFFF",
        "font_size": 60,
        "stroke_color": "#000000",
        "stroke_width": 1.5
    }

params = VideoParams(**params_dict)
task_id = 'task_id_test_123'

In [3]:
params.video_subject


'怎么快速赚到100万'

In [4]:
config.app.get("openai_model_name", "openai")

'gpt-3.5-turbo'

In [5]:
video_subject = params.video_subject
voice_name = voice.parse_voice_name(params.voice_name)
paragraph_number = params.paragraph_number
n_threads = params.n_threads
max_clip_duration = params.video_clip_duration
logger.info("\n\n## generating video script")
video_script = params.video_script.strip()
if not video_script:
    video_script = llm.generate_script(video_subject=video_subject, language=params.video_language,
                                       paragraph_number=paragraph_number)
else:
    logger.debug(f"video script: \n{video_script}")
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=10)

2024-05-09 23:34:01 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_81112/2255889761.py:6": <module> - 

## generating video script
2024-05-09 23:34:01 | INFO | "./app/services/llm.py:215": generate_script - subject: 怎么快速赚到100万
2024-05-09 23:34:01 | INFO | "./app/services/llm.py:16": _generate_response - llm provider: openai
2024-05-09 23:34:05 | SUCCESS | "./app/services/llm.py:248": generate_script - completed: 
首先，要确立明确的目标和计划。接着，要不断学习和提升自己的技能，以增加赚钱的机会。同时，要找到一个适合自己的赚钱方式，并全力以赴。还要注重在投资和理财方面的知识，以实现财务自由的目标。最后，要坚持不懈地努力，相信自己能够实现赚到100万的目标。


In [6]:
logger.info("\n\n## generating video terms")
video_terms = params.video_terms
if not video_terms:
    video_terms = llm.generate_terms(video_subject=video_subject, video_script=video_script, amount=5)
else:
    if isinstance(video_terms, str):
        video_terms = [term.strip() for term in re.split(r'[,，]', video_terms)]
    elif isinstance(video_terms, list):
        video_terms = [term.strip() for term in video_terms]
    else:
        raise ValueError("video_terms must be a string or a list of strings.")
    logger.debug(f"video terms: {utils.to_json(video_terms)}")
script_file = path.join(utils.task_dir(task_id), f"script.json")
script_data = {
    "script": video_script,
    "search_terms": video_terms,
    "params": params,
}
with open(script_file, "w", encoding="utf-8") as f:
    f.write(utils.to_json(script_data))
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=20)

2024-05-09 23:34:05 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_81112/447892559.py:1": <module> - 

## generating video terms
2024-05-09 23:34:05 | INFO | "./app/services/llm.py:279": generate_terms - subject: 怎么快速赚到100万
2024-05-09 23:34:05 | INFO | "./app/services/llm.py:16": _generate_response - llm provider: openai
2024-05-09 23:34:06 | SUCCESS | "./app/services/llm.py:300": generate_terms - completed: 
['quick money', 'financial goal', 'investment knowledge', 'money-making opportunities', 'wealth accumulation']


In [7]:
logger.info("\n\n## generating audio")
audio_file = path.join(utils.task_dir(task_id), f"audio.mp3")
sub_maker = voice.tts(text=video_script, voice_name=voice_name, voice_file=audio_file)
if sub_maker is None:
    sm.state.update_task(task_id, state=const.TASK_STATE_FAILED)
    logger.error(
        """failed to generate audio:
1. check if the language of the voice matches the language of the video script.
2. check if the network is available. If you are in China, it is recommended to use a VPN and enable the global traffic mode.
    """.strip()
    )
    
audio_duration = voice.get_audio_duration(sub_maker)
audio_duration = math.ceil(audio_duration)
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=30)

2024-05-09 23:34:06 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_81112/2105989780.py:1": <module> - 

## generating audio
2024-05-09 23:34:06 | INFO | "./app/services/voice.py:1051": azure_tts_v1 - start, voice name: zh-CN-XiaoxiaoNeural, try: 1
2024-05-09 23:34:06 | ERROR | "./app/services/voice.py:1073": azure_tts_v1 - failed, error: This event loop is already running
2024-05-09 23:34:06 | INFO | "./app/services/voice.py:1051": azure_tts_v1 - start, voice name: zh-CN-XiaoxiaoNeural, try: 2
2024-05-09 23:34:06 | ERROR | "./app/services/voice.py:1073": azure_tts_v1 - failed, error: This event loop is already running
2024-05-09 23:34:06 | INFO | "./app/services/voice.py:1051": azure_tts_v1 - start, voice name: zh-CN-XiaoxiaoNeural, try: 3
2024-05-09 23:34:06 | ERROR | "./app/services/voice.py:1073": azure_tts_v1 - failed, error: This event loop is already running
2024-05-09 23:34:06 | ERROR | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000

/Users/roper/work/auto_video_generator/app/services/voice.py:1073: RuntimeWarning: coroutine 'azure_tts_v1.<locals>._do' was never awaited
  logger.error(f"failed, error: {str(e)}")


AttributeError: 'NoneType' object has no attribute 'offset'